# Grid Fluid

Modeling fluid motion by solving the incompressible and inviscid Navier-Stokes equation on a uniform grid

Credit to Bo Zhu, on whose Dartmouth COSC89 course notes the following code has been based.

Additionally, credit to Robert Bridson from the University of British Columbia, whose Fluid Simulation Course Notes were invaluable in understanding the material. His course notes can be found here:\
https://www.cs.ubc.ca/~rbridson/fluidsimulation/fluids_notes.pdf

In [41]:
import taichi as ti
import taichi.math as tm

ti.init()

[Taichi] Starting on arch=x64


In [82]:
### Global Variables ###

substeps = 10
iters = 1
dt = 0.02 
res = 500
num_nodes = res * res
dx = 0.015

vel = ti.Vector.field(2, float, shape=(res, res)) # do I make the grid with less nodes?
vel_divs = ti.field(float, shape=(res, res))
vel_curls = ti.field(float, shape=(res, res))
grad_p = ti.Vector.field(2, float, shape=(res, res))
pressure = ti.field(float, shape=(res, res))
smoke_density = ti.field(float, shape=(res,res))
vor = ti.field(float, shape=(res,res))
N = ti.Vector.field(2, float, shape=(res, res))

src_pos = tm.vec2(0.5, 0.5)
src_vel = tm.vec2(1.5, 0)
src_rad = 0.1

colors = ti.Vector.field(3, float, shape=(res,res))

debug = False

# tester variables
ctr = ti.field(int, shape=())
smoke_check = ti.field(float, shape=())
vel_check = ti.Vector.field(2, float, shape=())
new_pos_check = ti.Vector.field(2, float, shape=())
base_vel = ti.Vector.field(2, float, shape=())

In [83]:
### Helper Functions ###

# @param - coordinates (i,j) of the node
# @return - gradient of p at node (i,j)
@ti.func 
def gradient_pressure(i:ti.i32, j:ti.i32):
    pressures = tm.vec2(pressure[i+1, j] - pressure[i-1, j], pressure[i, j+1] - pressure[i, j-1])
    return pressures / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - divergence of the velocity at grid node (i, j)
@ti.func
def divergence_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][0] - vel[i-1, j][0] + vel[i, j+1][1] - vel[i, j-1][1]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the curl of the velocity at grid node (i, j)
@ti.func
def curl_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][1] - vel[i-1, j][1] + vel[i, j+1][0] - vel[i, j-1][0]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the laplacian of the pressure at grid node (i, j)
@ti.func
def laplacian_pressure(i:ti.i32, j:ti.i32):
    return (pressure[i+1, j] + pressure[i-1, j] + pressure[i, j+1] + pressure[i, j-1] - 4 * pressure[i, j]) / (dx * dx)

# returns whether or not (row, col) is on a boundary
@ti.func
def isBoundary(row:ti.i32, col:ti.i32) -> bool:
    return (row == 0 or col == 0 or row == res-1 or col == res-1)

In [84]:
### Simulator Functions ###

# @param - x_p: vec2, where x_p = (x - u^(n+1/2)(x) * dt) (the "starting point")
# @return - the interpolated value for u*(x) at x_p
@ti.func
def interpolate(x_p):
    x = x_p[0]
    y = x_p[1]
    x1 = (int)(ti.floor(x))
    x2 = x1+1
    y1 = (int)(ti.floor(y))
    y2 = y1+1
    # confine values to be within grid
#     x1 = ti.max(x1, 0)
#     x1 = ti.min(x1, res-1)
#     x2 = ti.max(x1, 0)
#     x2 = ti.min(x1, res-1)
#     y1 = ti.max(x1, 0)
#     y1 = ti.min(x1, res-1)
#     y2 = ti.max(x1, 0)
#     y2 = ti.min(x1, res-1)
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * vel[x1, y1] + (x - x1)/(x2 - x1) * vel[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * vel[x1, y2] + (x - x1)/(x2 - x1) * vel[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    return f_xy

# same thing as function above, but uses the density matrix instead
@ti.func
def interpolate_density(x_p):
    x = x_p[0]
    y = x_p[1]
    x1 = (int)(ti.floor(x))
    x2 = x1+1
    y1 = (int)(ti.floor(y))
    y2 = y1+1
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * smoke_density[x1, y1] + (x - x1)/(x2 - x1) * smoke_density[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * smoke_density[x1, y2] + (x - x1)/(x2 - x1) * smoke_density[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    return f_xy

# Advection using a semi-Lagrangian method
@ti.kernel
def advect():
    for row in range(res):
        for col in range(res):
            pos = tm.vec2(row, col)
            new_pos = pos - vel[row, col] * 0.5 * dt
            new_vel = interpolate(new_pos)
            new_pos = pos - new_vel * dt
            new_vel = interpolate(new_pos)
            vel[row, col] = new_vel
            new_den = interpolate_density(new_pos)
            smoke_density[row, col] = new_den
            
            if debug and row == 250 and col == 200:
                new_pos_check[None] = new_pos
                vel_check[None] = new_vel
                smoke_check[None] = new_den
    
@ti.func
def calc_divs():
    for row in range(res):
        for col in range(res):
            vel_divs[row, col] = 0 # do I need this?
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)
                coef_off_dia = 1 / (dx * dx)
                coef_dia = 4 / (dx * dx)
                off_diagonal = 0.
                off_diagonal += pressure[row-1, col] + pressure[row+1, col] + pressure[row, col+1] + pressure[row, col-1]
                pressure[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
        
@ti.func
def update_pressures():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                grad_p[row,col] = gradient_pressure(row, col)
        
@ti.func
def correct_vel():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel[row, col] -= grad_p[row, col]
    
@ti.kernel
def project():
    # Calc velocity for RHS of Poissan (div u*)
    calc_divs()
    # correct velocity with pressure gradient
    for i in range(iters):
        update_pressures()
    correct_vel()

@ti.func
def clear_vorticity():
    for i in ti.grouped(vor):
        vor[i] = 0

@ti.func
def update_vorticity_field():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vor[row, col] = curl_vel(row, col)

@ti.func
def update_N():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                N[row, col][0] = abs(vor[row+1, col]) - abs(vor[row-1, col])
                N[row, col][1] = abs(vor[row, col+1]) - abs(vor[row, col-1])
                N[row, col].normalized()

@ti.func
def update_velocity():
    vor_conf_coef = 8
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                f = vor_conf_coef * dx * tm.cross(tm.vec3(N[row, col][0], N[row, col][1], 0), tm.vec3(0,0,vor[row, col])) # How does this work? N is a 2d vector, and vor is a scalar
                f_2d = tm.vec2(f[0], f[1])
                vel[row, col] += f_2d * dt

@ti.kernel
def vorticity_confinement():
    clear_vorticity()
    update_vorticity_field()
    update_N() # update N (direction of local rotation)
    update_velocity() # find vorticity confinement force and update velocity

# create a "source" for the smoke
@ti.kernel
def source():
    # iterate over each grid node
    # if it's within "source_radius" of "source_pos", vel at that node is src_vel, smoke_density there is 1
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - src_pos).norm() < src_rad:
                vel[row, col] = src_vel
                smoke_density[row, col] = 1
                ctr[None] += 1

@ti.kernel
def update_colors():
    for i in ti.grouped(colors):
        colors[i] = tm.vec3(smoke_density[i], smoke_density[i], smoke_density[i])

In [85]:
def substep():
    # source
#     source()
    # swap new_velocity and velocity to update the n+1 as n for this time step
    
    advect()
    project()
    update_colors()
#     vorticity_confinement()

In [86]:
### Setup ###
@ti.kernel
def init_grid():
    for i in ti.grouped(vel):
        vel[i] = tm.vec2(0,0)
        grad_p[i] = tm.vec2(0,0)
        N[i] = tm.vec2(0,0)
    for i in ti.grouped(smoke_density):
        smoke_density[i] = 0
        vel_divs[i] = 0
        vel_curls[i] = 0
        pressure[i] = 0
        vor[i] = 0

    base_vel[None] = vel[250,250]
#     smoke_check[None] = interpolate_density(tm.vec2(1,1))
    ctr[None] = 0

In [ ]:
### Driver ###

def main():
    # gui = ti.GUI('Grid Fluid', (res, res))
    gui = ti.GUI('Grid Fluid', (res, res), fast_gui=True)
        
    init_grid()
    source()

    if debug:
        for step in range(substeps):
            print("step ", step)
            substep()
            print("smoke check", smoke_check[None])
            print("smoke density at middle", smoke_density[250, 250])
            print("vel check after one interpolation", vel_check[None])
            print("new pos check", new_pos_check[None])
            print("base vel", base_vel[None])
        
    while gui.running:
        
        # Move stuff
        if not debug:
            for step in range(substeps): # do I need this?
                substep()
             
        # Draw grid
        gui.set_image(colors)
        gui.show()
        
    
if __name__ == '__main__':
    main()